<a href="https://colab.research.google.com/github/daksh0103-prog/DREAM-STORY-RAG/blob/main/Movie_Chatbot_Main_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# pip install requests transformers

import requests
from transformers import pipeline

# Load question-answering model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Your TMDb API Key
TMDB_API_KEY = "fd579f8ceb50584ca08c356d6e0416ea"  # 🔑 Replace this with your real API key

def search_movie_on_tmdb(query):
    url = "https://api.themoviedb.org/3/search/multi"  # supports movie + tv
    params = {
        "api_key": TMDB_API_KEY,
        "query": query
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])

    # Prefer TV or movie results
    for result in results:
        if result["media_type"] in ["movie", "tv"]:
            return result
    return None

def get_movie_details(media_type, media_id):
    url = f"https://api.themoviedb.org/3/{media_type}/{media_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "append_to_response": "credits,reviews"
    }
    response = requests.get(url, params=params)
    return response.json()

def build_context(movie_data, media_type):
    title = movie_data.get("title") or movie_data.get("name", "")
    overview = movie_data.get("overview", "")
    director = ""
    cast = []

    if media_type == "movie":
        for crew_member in movie_data.get("credits", {}).get("crew", []):
            if crew_member["job"] == "Director":
                director = crew_member["name"]
                break
    elif media_type == "tv":
        # TV series don't always have "Director" in credits
        director = movie_data.get("created_by", [{}])[0].get("name", "")

    cast_list = movie_data.get("credits", {}).get("cast", [])[:5]
    cast = [actor["name"] for actor in cast_list]

    context = f"Title: {title}\nOverview: {overview}\nCreator/Director: {director}\nCast: {', '.join(cast)}"
    return context

def get_movie_answer(user_question):
    try:
        movie = search_movie_on_tmdb(user_question)
        if not movie:
            return "I couldn't find that movie or show. Try another title."

        media_type = movie["media_type"]  # 'movie' or 'tv'
        movie_details = get_movie_details(media_type, movie["id"])
        context = build_context(movie_details, media_type)

        result = qa_pipeline(question=user_question, context=context)
        return result["answer"]

    except Exception as e:
        return f"Error: {e}"

def chat():
    print("🎬 MovieBot: Ask me about any movie or TV show! (Type 'exit' to quit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("MovieBot: Goodbye! 🍿")
            break
        response = get_movie_answer(user_input)
        print(f"MovieBot: {response}")

chat()


Device set to use cpu


🎬 MovieBot: Ask me about any movie or TV show! (Type 'exit' to quit)
You: Fast and Furious
MovieBot: Fast and Furious" (1924), a fast-paced silent comedy
You: exit
MovieBot: Goodbye! 🍿
